# 03 Data Ingestion via ADF - Data Transformations Notebook

This notebook performs the data munging and transformation tasks against the files ingested via ADF in the Data Ingestion notebook for this module. This notebook will be executed by a call from Azure Data Factory (ADF), and will automate the process of data munging and table creation for the data used within this module.

The following table will be created by this notebook:

- homicides_2016

In [3]:
# Create input widgets, which will accept parameters passed in via the ADF Databricks Notebook activity
dbutils.widgets.text("accountName", "", "Account Name")
dbutils.widgets.text("accountKey", "", "Account Key")
dbutils.widgets.text("containerName", "", "Container Name")

In [4]:
# Assign variables to the passed in values of the widgets
accountName = dbutils.widgets.get("accountName")
accountKey = dbutils.widgets.get("accountKey")
containerName = dbutils.widgets.get("containerName")

# Create connection string to use for accessing files in the storage account
connectionString = "wasbs://%(containerName)s@%(accountName)s.blob.core.windows.net/03.02" % locals()

In [5]:
# Create connection to the Azure Storage account
spark.conf.set("fs.azure.account.key." + accountName + ".blob.core.windows.net", accountKey)

### Create DataFrame for each city's crime data

In [7]:
bostonDf = spark.read.parquet("%(connectionString)s/Crime-Data-Boston-2016.parquet" % locals())
chicagoDf = spark.read.parquet("%(connectionString)s/Crime-Data-Chicago-2016.parquet" % locals())
dallasDf = spark.read.parquet("%(connectionString)s/Crime-Data-Dallas-2016.parquet" % locals())
losAngelesDf = spark.read.parquet("%(connectionString)s/Crime-Data-Los-Angeles-2016.parquet" % locals())
newOrleansDf = spark.read.parquet("%(connectionString)s/Crime-Data-New-Orleans-2016.parquet" % locals())
newYorkDf = spark.read.parquet("%(connectionString)s/Crime-Data-New-York-2016.parquet" % locals())
phillyDf = spark.read.parquet("%(connectionString)s/Crime-Data-Philadelphia-2016.parquet" % locals())

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1395183622383482> in <module> () 
 ----> 1 bostonDf = spark . read . parquet ( "%(connectionString)s/Crime-Data-Boston-2016.parquet" % locals ( ) ) 
 2 chicagoDf = spark . read . parquet ( "%(connectionString)s/Crime-Data-Chicago-2016.parquet" % locals ( ) ) 
 3 dallasDf = spark . read . parquet ( "%(connectionString)s/Crime-Data-Dallas-2016.parquet" % locals ( ) ) 
 4 losAngelesDf = spark . read . parquet ( "%(connectionString)s/Crime-Data-Los-Angeles-2016.parquet" % locals ( ) ) 
 5 newOrleansDf = spark . read . parquet ( "%(connectionString)s/Crime-Data-New-Orleans-2016.parquet" % locals ( ) ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths) 
 314 [ ( 'name' , 'string' ) , ( 'year' , 'int' ) , ( 'month' , 'int' ) , ( 'day' , 'int' ) ] 
 315 """
 --> 316 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 317 
 318 @ ignore_unicode_prefix 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o297.parquet.
: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: java.lang.IllegalArgumentException: URI 'wasbs://@.blob.core.windows.net/03.02/Crime-Data-Boston-2016.parquet' has a malformed WASB authority, expected container name.Authority takes the form wasb://[<container name>@]<account name>
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.createAzureStorageSession(AzureNativeFileSystemStore.java:1031)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.initialize(AzureNativeFileSystemStore.java:482)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.initialize(NativeAzureFileSystem.java:1279)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:370)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at com.databricks.sql.transaction.tahoe.DeltaTableUtils$.findDeltaTableRoot(DeltaTable.scala:96)
	at org.apache.spark.sql.DataFrameReader.preprocessDeltaLoading(DataFrameReader.scala:226)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:258)
	at org.apache.spark.sql.DataFrameReader.parquet(DataFrameReader.scala:724)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: URI 'wasbs://@.blob.core.windows.net/03.02/Crime-Data-Boston-2016.parquet' 

### Create normalized DataFrames for each city

For the upcoming aggregation, you need to alter the data sets to include a `month` column which can be computed from the various "Incident Date" columns using the `month()` function. Boston already has this column.

In this example, we use several functions in the `pyspark.sql.functions` library, and need to import:

* `col()` to return a column from a DataFrame, based on the given column name.
* `contains(mySubstr)` to indicate a string contains substring `mySubstr`.
* `lit()` to create a column from a literal value.
* `lower()` to convert text to lowercase.
* `month()` to extract the month from `reportDate` timestamp data type.
* `unix_timestamp()` to convert the Dallas date field into a timestamp format, so the month can be extracted using the `month()` function.

Also, note:

* We use  `|`  to indicate a logical `or` of two conditions in the `filter` method.
* We use `lit()` to create a new column in each DataFrame containing the name of the city for which the data is derived.

In [9]:
# Import required libraries
import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import col, lit, lower, month, unix_timestamp, upper

In [10]:
homicidesBostonDf = (bostonDf.withColumn("city", lit("Boston"))
  .select("month", col("OFFENSE_CODE_GROUP").alias("offense"), col("city"))
  .filter(lower(col("OFFENSE_CODE_GROUP")).contains("homicide"))
)

In [11]:
homicidesChicagoDf = (chicagoDf.withColumn("city", lit("Chicago"))
  .select(month(col("date")).alias("month"), col("primaryType").alias("offense"), col("city"))
  .filter(lower(col("primaryType")).contains("homicide"))
)

In [12]:
homicidesDallasDf = (dallasDf.withColumn("city", lit("Dallas"))
   .select(month(unix_timestamp(col("callDateTime"),"M/d/yyyy h:mm:ss a").cast("timestamp")).alias("month"), col("typeOfIncident").alias("offense"), col("city"))
   .filter(lower(col("typeOfIncident")).contains("murder") | lower(col("typeOfIncident")).contains("manslaughter"))
)

In [13]:
homicidesLosAngelesDf = (losAngelesDf.withColumn("city", lit("Los Angeles"))
   .select(month(col("dateOccurred")).alias("month"), col("crimeCodeDescription").alias("offense"), col("city"))
   .filter(lower(col("crimeCodeDescription")).contains("homicide") | lower(col("crimeCodeDescription")).contains("manslaughter"))
)

In [14]:
homicidesNewOrleansDf = (newOrleansDf.withColumn("city", lit("New Orleans"))
   .select(month(col("Occurred_Date_Time")).alias("month"), col("Incident_Description").alias("offense"), col("city"))
   .filter(lower(col("Incident_Description")).contains("homicide") | lower(col("Incident_Description")).contains("murder"))
)

In [15]:
homicidesNewYorkDf = (newYorkDf.withColumn("city", lit("New York"))
  .select(month(col("reportDate")).alias("month"), col("offenseDescription").alias("offense"), col("city")) 
  .filter(lower(col("offenseDescription")).contains("murder") | lower(col("offenseDescription")).contains("homicide"))
)

In [16]:
homicidesPhillyDf = (phillyDf.withColumn("city", lit("Philadelphia"))
   .select(month(col("dispatch_date")).alias("month"), col("text_general_code").alias("offense"), col("city"))
   .filter(lower(col("text_general_code")).contains("homicide"))
)

### Create a single DataFrame
With the normalized homicide data for each city, combine the two by taking their union.

In [18]:
homicidesDf = homicidesNewYorkDf.union(homicidesBostonDf).union(homicidesChicagoDf).union(homicidesDallasDf).union(homicidesLosAngelesDf).union(homicidesNewOrleansDf).union(homicidesPhillyDf)

### Export the prepared data to a persistent table

In [20]:
homicidesDf.write.mode("overwrite").saveAsTable("homicides_2016")

### Return OK status

In [22]:
import json
dbutils.notebook.exit(json.dumps({
  "status": "OK",
  "message": "Cleaned data and created persistent table",
  "tables": ["homicides_2016"]
}))

{"tables": ["homicides_2016"], "status": "OK", "message": "Cleaned data and created persistent table"}